# Sizing script for a rocket engine

## Introduction
This script is used to size the Eve engine.
The script is designed to be used in a Jupyter Notebook.

## Running the script
To run the script, simply run all cells in the notebook.

## Imports
Let's start by importing the necessary libraries.

In [42]:
from math import sqrt
import cantera as ct
import matplotlib.pyplot as plt

## Constants

In [43]:
g0 = 9.80665  # m/s^2
R = 287.05  # J/kg/K
gamma = 1.4  # Cp/Cv

## Design parameters
The next cell contains the design parameters for the engine.
We start from a thrust target and a chamber pressure target.

In [44]:
# Main design parameters
thrust_target = 1000  # N
chamber_pressure = 1 * 1e6  # Pa
# I_sp = 260  # s

# Nozzle parameters
atm_pressure = 101325  # Pa
exit_pressure = atm_pressure  # Pa

# Propellant parameters
mixture_ratio = 2  # oxidizer/fuel mass ratio
fuel_species = 'CH4'
oxidizer_species = 'O2'
propellant_temperature = 300  # K

## Propellant exit conditions

In [45]:
gas = ct.Solution('gri30.yaml')
gas.TP = propellant_temperature, chamber_pressure
gas.set_equivalence_ratio(phi=2, fuel=fuel_species, oxidizer=oxidizer_species)
print('Gas properties before combustion:')
print(gas.report())
gas.equilibrate('HP')

# Get properties from gas object
print('Gas properties after combustion:')
print(gas.report())

# Calculate exhaust gamma
gamma_exhaust = gas.cp / gas.cv

print(f"Adiabatic flame temperature: {gas.T:.1f} K")

fuel_density = gas.density * gas[fuel_species].molecular_weights[0]  # kg/m^3
oxidizer_density = gas.density * gas[oxidizer_species].molecular_weights[0]  # kg/m^3

# Calculate exit velocity
exit_velocity = sqrt(gamma_exhaust * R * gas.T * (1 - (exit_pressure / chamber_pressure)**((gamma_exhaust - 1) / gamma_exhaust)) / (gamma_exhaust - 1))
print(f"Exit velocity: {exit_velocity:.1f} m/s")

# Calculate Isp
I_sp = exit_velocity / g0
print(f"Isp: {I_sp:.1f} s")

# Calculate propellant composition
exhaust_composition = gas.X
print('Exhaust composition:')
for i, species in enumerate(gas.species_names):
    if exhaust_composition[i] > 0.001:
        print(f'{species}: {exhaust_composition[i]:.3f}')

Gas properties before combustion:

  gri30:

       temperature   300 K
          pressure   1e+06 Pa
           density   9.63 kg/m^3
  mean mol. weight   24.02 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -1.5503e+06        -3.724e+07  J
   internal energy       -1.6542e+06       -3.9734e+07  J
           entropy            7605.5        1.8269e+05  J/K
    Gibbs function        -3.832e+06       -9.2046e+07  J
 heat capacity c_p            1356.1             32574  J/K
 heat capacity c_v              1010             24260  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                O2           0.66606               0.5           -23.077
               CH4           0.33394               0.5           -50.727
     [  +51 minor]                 0           

## Flow rates
The mass flow rate is calculated from the thrust and the chamber pressure.

In [46]:
mass_flow_rate = thrust_target / I_sp / g0
print(f"Mass flow rate: {mass_flow_rate:.3f} kg/s")

# Calculate oxidizer/fuel mass ratio
fuel_mass_flow_rate = mass_flow_rate / (mixture_ratio + 1)
oxidizer_mass_flow_rate = fuel_mass_flow_rate * mixture_ratio

print(f"Oxidizer mass flow rate: {oxidizer_mass_flow_rate:.3f} kg/s \t {oxidizer_mass_flow_rate*1000:.3f} g/s")
print(f"Fuel mass flow rate: {fuel_mass_flow_rate:.3f} kg/s \t {fuel_mass_flow_rate*1000:.3f} g/s")

# Calculate oxidizer/fuel volume ratio
oxidizer_volume_flow_rate = oxidizer_mass_flow_rate / oxidizer_density
fuel_volume_flow_rate = fuel_mass_flow_rate / fuel_density

print(f"Oxidizer volume flow rate: {oxidizer_volume_flow_rate:.6f} m^3/s \t {oxidizer_volume_flow_rate*1000:.3f} L/s")
print(f"Fuel volume flow rate: {fuel_volume_flow_rate:.6f} m^3/s \t {fuel_volume_flow_rate*1000:.3f} L/s")

Mass flow rate: 0.835 kg/s
Oxidizer mass flow rate: 0.557 kg/s 	 556.953 g/s
Fuel mass flow rate: 0.278 kg/s 	 278.476 g/s
Oxidizer volume flow rate: 0.024553 m^3/s 	 24.553 L/s
Fuel volume flow rate: 0.024485 m^3/s 	 24.485 L/s


## Nozzle dimensions

In [48]:
w_t = mass_flow_rate
print(f"w_t: {w_t:.3f}")

M = 1

T_t = gas.T * (1 + (gamma_exhaust - 1) / 2) * M ** 2
print(f"Throat temperature: {T_t:.1f} K")

p_t = gas.P * (1 + M ** 2 * (gamma_exhaust - 1) / 2) ** (gamma_exhaust / (gamma_exhaust - 1))
print(f"Throat pressure: {p_t:.1f} Pa")

throat_area = w_t / (p_t * M * sqrt(gamma_exhaust / R / T_t))
print(f"Throat area: {throat_area:.6f} m^2")
print(f"Throat diameter: {sqrt(throat_area)*1000:.3f} mm")

chamber_area = 10 * throat_area
print(f"Chamber area: {chamber_area:.6f} m^2")
print(f"Chamber diameter: {sqrt(chamber_area)*1000:.3f} mm")

w_t: 0.835
Throat temperature: 3022.8 K
Throat pressure: 1795966.7 Pa
Throat area: 0.000389 m^2
Throat diameter: 19.726 mm
Chamber area: 0.003891 m^2
Chamber diameter: 62.379 mm
